In [1]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import StratifiedShuffleSplit

from threading import Thread, Lock

## Dataset Configuration

In [2]:
d_clinical = pd.read_csv(os.path.join('data', 'dataset_clinical_cleaned.csv'))
d_genetic = pd.read_csv(os.path.join('data', 'dataset_genetic_cleaned_noOHE.csv'))
d_vampire = pd.read_csv(os.path.join('data', 'dataset_vampire_cleaned.csv'))
outputs = pd.read_csv(os.path.join('data', 'outputs_cleaned.csv'))

In [3]:
C = d_clinical.values
G = d_genetic.values
V = d_vampire.values

In [4]:
y_d = outputs["dement_fail"].values
y_c = outputs["cvd_fail"].values

In [5]:
tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25).split(C, y_d))

In [6]:
C_ = C[tr_idx]
C_test = C[ts_idx]
G_ = G[tr_idx]
G_test = G[ts_idx]
V_ = V[tr_idx]
V_test = V[ts_idx]

In [7]:
y_d_ = y_d[tr_idx]
y_d_test = y_d[ts_idx]

y_c_ = y_c[tr_idx]
y_c_test = y_c[ts_idx]

In [8]:
ds_list = [C_, G_, V_]
ds_test = [C_test, G_test, V_test]
ds_names = ['clinic', 'genetic', 'vampire']

## Kernel Definition

In [9]:
kernel_names_0 = ['linear', 'polynomial', 'gaussian']
kernel_type_0 = [{'linear':[0.5], 'polynomial':[2, 3, 7], 'gaussian':[0.1, 0.5, 0.7]},
               {'linear':[0.2], 'polynomial':[4, 5, 8], 'gaussian':[0.7, 1]}]

In [10]:
kernel_names_1 = ['sigmoid', 'polynomial', 'gaussian']
kernel_type_1 = [{'sigmoid':[0.1, 0.2, 0.3], 'polynomial':[2, 3], 'gaussian':[0.2, 0.3]},
               {'sigmoid':[0.5, 0.7], 'polynomial':[5, 8], 'gaussian':[0.5, 0.7]}]

## Other shared parameters initialization

In [11]:
estimator = ca.centeredKernelAlignment
lock_dementia = Lock()
lock_cardio = Lock()
valid_fold = 3
threads = []
pen_params = [0.5, 0.7, 1.3]

## Thread

In [12]:
def child(sampler,estimator,ds_list,ds_test,y_d_,y_d_test,y_c_,y_c_test,lock_dementia,lock_cardio,valid_fold,exclusion_list,verbose,approach):

    #DEMENTIA
    
    #Linear - Polynomial - Gaussian
    result1 = sampler.sample(kernel_type_0, estimator, ds_list, y_d_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_0)
    ut.testConfigurations(estimator, y_d_, y_d_test, w_list, ds_list, ds_test, kernel_names_0, lamb_list, sparsity, 'classification', lock_dementia, fileToWrite = 'results_temp/Dementia_test_ClinicalBestToy.txt', header = 'Dementia Linear - Polynomial - Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Dementia_train_ClinicalBestToy.txt', header = '\nDementia Linear - Polynomial - Gaussian\n' + approach + '\n', lock = lock_dementia)
    
    #Sigmoid - Polynomial - Gaussian
    result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_d_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_1)
    ut.testConfigurations(estimator, y_d_, y_d_test, w_list, ds_list, ds_test, kernel_names_1, lamb_list, sparsity, 'classification', lock_dementia, fileToWrite = 'results_temp/Dementia_test_ClinicalBestToy.txt', header = 'Dementia Sigmoid - Polynomial - Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Dementia_train_ClinicalBestToy.txt.txt', header = '\nDementia Sigmoid - Polynomial - Gaussian\n' + approach + '\n', lock = lock_dementia)

    
    # CARDIO
    
    #Linear - Polynomial - Gaussian
    result1 = sampler.sample(kernel_type_0, estimator, ds_list, y_c_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_0)
    ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_0, lamb_list, sparsity, 'classification', lock_cardio, fileToWrite = 'results_temp/Cardio_test_ClinicalBestToy.txt', header = 'Cardio Linear - Polynomial - Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Cardio_train_ClinicalBestToy.txt.txt', header = '\nCardio Linear - Polynomial - Gaussian\n' + approach + '\n', lock = lock_cardio)
    
    
    #Sigmoid - Polynomial - Gaussian
    result1 = sampler.sample(kernel_type_1, estimator, ds_list, y_c_, valid_fold = valid_fold, verbose=verbose)
    w_dict, w_list, lamb_list, sparsity = result1.votingOverCA(ds_names, kernel_names_1)
    ut.testConfigurations(estimator, y_c_, y_c_test, w_list, ds_list, ds_test, kernel_names_1, lamb_list, sparsity, 'classification', lock_cardio, fileToWrite = 'results_temp/Cardio_test_ClinicalBestToy.txt', header = 'Cardio Sigmoid - Polynomial - Gaussian \n' + approach + '\n', normalize = sampler.normalize_kernels, verbose=verbose)
    result1.performancesFeatures(fileToWrite = 'results_temp/Cardio_train_ClinicalBestToy.txt.txt', header = '\nCardio Sigmoid - Polynomial - Gaussian\n' + approach + '\n', lock = lock_cardio)
    

## L2 Penalty, Centering, Normalization

In [13]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = False, centering = True, normalizing = True)

child(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []], False, 'L2 - Centering - Normalizing')

## L2 Penalty, Centering, K-Normalization

In [14]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=0.25, sparsity = False, centering = True, normalize_kernels = True)

child(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []], False, 'L2 - Centering - K Normalizing')

## L1 Penalty, Centering, Normalization

In [15]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalizing = True)

child(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []], False, 'L1 - Centering - Normalizing')

/home/students/Documents/BCB_Project/myLasso3.py:24: RuntimeWarning: invalid value encountered in true_divide
  eta_new /= np.linalg.norm(eta_new)


## L1 Penalty, Centering, K-Normalization

In [16]:
sampler = ms.mySampleWrapper(pen_params, n_splits=3, test_size=.25, sparsity = True, centering = True, normalize_kernels = True)

child(sampler, estimator, ds_list, ds_test, y_d_, y_d_test, y_c_, y_c_test, lock_dementia, lock_cardio, valid_fold, [[5,6,7,9,10,13,15,16,17,18,19], list(range(G.shape[1]-3)), []], False, 'L1 - Centering - K Normalizing')

/home/students/Documents/BCB_Project/myLasso3.py:24: RuntimeWarning: invalid value encountered in true_divide
  eta_new /= np.linalg.norm(eta_new)


KeyboardInterrupt: 